In [1]:
# pip install polytope
# pip install pypoman

import math
import numpy as np
from sympy.geometry import Point
from pypoman import compute_polytope_halfspaces, compute_polytope_vertices
from pypoman.duality import convex_hull
from polytope import polytope
from polytope.polytope import enumerate_integral_points, qhull, box2poly, cheby_ball
from scipy.spatial import ConvexHull

Let's construct a reflexive 2D polytope (taken from the book "The Calabi–Yau Landscape", page 44)

In [2]:
V = np.array([[0, -1], [-1, 0], [-1, 2], [1, 0], [1, -1]])

Pypoman provides a form $0\le -Ax+b$

In [3]:
A, b = compute_polytope_halfspaces(V)

print(A)
print(b)

print(compute_polytope_vertices(A, b))
print(convex_hull(V))

[[-0. -1.]
 [-1. -0.]
 [-1. -1.]
 [ 1.  1.]
 [ 1. -0.]]
[1. 1. 1. 1. 1.]
[array([-1.,  2.]), array([1., 0.]), array([ 1., -1.]), array([ 0., -1.]), array([-1.,  0.])]
[array([-1,  2]), array([-1,  0]), array([ 0, -1]), array([ 1, -1]), array([1, 0])]


Polytope provides a form $Ax\le b$ which is equivalent to pypoman's form $0\le-Ax+b$

In [4]:
P = qhull(V)

print(P.A)
print(P.b)
print(cheby_ball(P))
#print(P.vertices)

[[-1.       0.     ]
 [ 1.      -0.     ]
 [ 0.70711  0.70711]
 [-0.70711 -0.70711]
 [ 0.      -1.     ]]
[1.      1.      0.70711 0.70711 1.     ]
(0.7071067811865475, array([0., 0.]))


Scipy yields an array $[normal, offset]$ that describes $Ax+b\le0$ forming the hyperplane equation of the facet see [documentation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.ConvexHull.html)

In [5]:
hull = ConvexHull(V)
print(hull.equations)

[[-1.       0.      -1.     ]
 [ 0.70711  0.70711 -0.70711]
 [-0.70711 -0.70711 -0.70711]
 [ 1.       0.      -1.     ]
 [-0.      -1.      -1.     ]]


When using the negated form of pypoman's result, that is $Ax-b\le0$ then $b$ is the distance we are looking for.

The distances are defined in the hyperplane representation. The hyperplanes are given as $<u,v> \ge a$ where $u$ are points in the polytope lattice $M, v$ is a single point in the dual lattice $N$ and $a$ is the distance which is real number. If we just get the hyperplane representation then we automatically have the distances.

In [6]:
A, b = compute_polytope_halfspaces(V)
print(b)

[1. 1. 1. 1. 1.]


Count integral points using polytope library.
An interesting reference is also [Effective lattice point counting in rational convex polytopes](https://www.sciencedirect.com/science/article/pii/S0747717104000422?via%3Dihub)

In [7]:
V = np.array([[0, -1], [-1, 0], [-1, 2], [1, 0], [1, -1]])
poly_qhull = qhull(V)
integral_points = enumerate_integral_points(poly_qhull)
integral_points = integral_points.transpose()
integral_points = integral_points.astype(int)
print(integral_points)

[[ 0 -1]
 [ 1 -1]
 [-1  0]
 [ 0  0]
 [ 1  0]
 [-1  1]
 [ 0  1]
 [-1  2]]


The fitness function consists of two parts.

The first term is IP(Delta)-1, where IP(Delta) = 1 if Delta satisfies IP and 0 otherwise, adds a penalty if Delta doesn’t satisfy the IP property.

The second term sum(ai-1), where ai are the hyperplane distances of each face, adds a penalty if the distances of all the hyper planes aren’t at a distance 1 from the origin.

Note: Ensure that when you generate polytopes that you choose its points around the origin! Otherwise you have to translate the polytope such that its origin or one of its interior points becomes $(0,0,0,\ldots)$.

In [8]:
#
# 2D Example
#
#V = np.array([[0, -1], [-1, 0], [-1, 2], [1, 0], [1, -1]])

#
# 3D Example
#
#good cube: its origin is (1,1,1)
#V = np.array([[0,0,0], [2,0,0], [2,2,0], [0,2,0], [0,2,2], [2,2,2], [2,0,2], [0,0,2]])

#good cube: its origin is (0,0,0)
#V = np.array([[-1,-1,-1], [1,-1,-1], [1,1,-1], [-1,1,-1], [-1,1,1], [1,1,1], [1,-1,1], [-1,-1,1]])

#
# 5D Example
#
#V = np.array([[-1,0,1,1,-1], [0,1,-1,-1,1], [0,-3,-1,1,0], [-1,0,0,-1,0], [-1,1,-1,-1,1], [0,-1,1,1,0], [0,0,0,-1,0]])
V = np.array([[-2,0,1,0,-1], [1,0,1,-1,2], [-1,1,-1,-2,0], [1,-1,0,2,0], [1,0,0,-1,0], [0,0,0,2,-1]])

A, b = compute_polytope_halfspaces(V)

poly_qhull = qhull(V)

def grid_region(polyreg, res=None):
    # grid corners
    bbox = polyreg.bounding_box
    # grid resolution
    if res is None:
        density = 8
        res = [
            math.ceil(density * (b - a))
            for a, b in zip(*bbox)]
    if len(res) != polyreg.dim:
        raise ValueError((
            "`len(res)` must equal the polytope's dimension "
            "(which is {dim}), but instead `res` is:  {res}"
            ).format(dim=polyreg.dim, res=res))
    if any(n < 1 for n in res):
        raise ValueError((
            '`res` must contain `int` values >= 1, '
            'instead `res` equals:  {res}'
            ).format(res=res))
    linspaces = list()
    for a, b, n in zip(*bbox, res):
        r = np.linspace(a, b, num=n)
        linspaces.append(r)
    points = np.meshgrid(*linspaces)
    x = np.vstack(list(map(np.ravel, points)))
    x = x[:, polyreg.contains(x, abs_tol=0)]
    return (x, res)

def enumerate_stricktly_integral_points(poly):
    a, b = poly.bounding_box
    a_int = np.floor(a)
    b_int = np.ceil(b)
    intervals = list(zip(a_int.flatten(), b_int.flatten()))
    box = box2poly(intervals)
    res = [int(b - a + 1) for a, b in intervals]
    grid, _ = grid_region(box, res=res)
    inside = poly.contains(grid, abs_tol=0)
    return grid[:, inside]

integral_points = enumerate_integral_points(poly_qhull)
integral_points = integral_points.transpose()
integral_points = integral_points.astype(int)

stricktly_integral_points = enumerate_stricktly_integral_points(poly_qhull)
stricktly_integral_points = stricktly_integral_points.transpose()
stricktly_integral_points = stricktly_integral_points.astype(int)
num_stricktly_integral_points = len(stricktly_integral_points)

print("integral_points: {0}".format(integral_points.tolist()))
print("stricktly_integral_points: {0}".format(stricktly_integral_points.tolist()))
print("b: {0}".format(b))

def fitness(ip_count, distances):
    result = 0
    if ip_count > 1:
        result -= 1
    for d in distances:
        result -= abs(d-1)
    return result

print("fitness value: {0}".format(fitness(num_stricktly_integral_points, b)))

integral_points: [[1, -1, 0, 2, 0], [-2, 0, 1, 0, -1], [0, 0, 0, 0, 0], [0, 0, 0, 2, -1], [1, 0, 0, -1, 0], [1, 0, 1, -1, 2], [-1, 1, -1, -2, 0]]
stricktly_integral_points: [[0, 0, 0, 0, 0]]
b: [1. 1. 1. 1. 1. 1.]
fitness value: 0.0


Some experimental stuff starts here:

In [9]:
import numpy as np
import polytope

# halfspace representation computed using `polytope`
# from the vertex representation given in the question
vertices = np.array([[0, -1], [-1, 0], [-1, 2], [1, 0], [1, -1]])
poly = polytope.qhull(vertices)
# first halfspace representation
A = np.array([
    [0, -1],
    [-1, 0],
    [-1, -1],
    [1, 1],
    [1, 0]])
b = np.array([1, 1, 1, 1, 1])
question_poly_1 = polytope.Polytope(A, b)
# second halfspace representation
A = np.array([
    [-0.70711, -0.70711],
    [-1, -0],
    [0.70711, 0.70711],
    [0, -1],
    [1, 0]])
b = np.array([0.70711, 1, 0.70711, 1, 1])

question_poly_2 = polytope.Polytope(A, b)
# check that all the above halfspace representations
# represent the same polytope
assert poly == question_poly_1, (poly, question_poly_1)
assert poly == question_poly_2, (poly, question_poly_2)

In [10]:
inner_point = np.array([0,0]) # in this case it is the origin
fac1 = polytope.quickhull.Facet([[0, -1], [-1, 0]])
fac2 = polytope.quickhull.Facet([[-1, 2], [1, 0]])
fac3 = polytope.quickhull.Facet([[1, -1], [0, -1]])

d1 = polytope.quickhull.distance(inner_point, fac1)
d2 = polytope.quickhull.distance(inner_point, fac2)
d3 = polytope.quickhull.distance(inner_point, fac3)
print(d1)
print(d2)
print(d3)

[-0.70711]
[-0.70711]
[-1.]
